In [ ]:
!pip install tensorflow-gpu==1.8.0 Keras==2.1.6

In [ ]:
!sudo apt-get install graphviz -y


In [ ]:
!pip install pydot


In [6]:
from keras.utils.vis_utils import model_to_dot
import pydot

import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
import itertools
import shutil
import os
import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
from IPython.display import SVG


In [2]:
!ls data

train  valid


In [3]:
train_folder = "data/train"
valid_folder = "data/valid"

In [4]:

IMAGE_SIZE=224
BATCH_SIZE=64
NUM_CLASSES=120

In [ ]:
!ls data/train/ | head

In [ ]:

train_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input
                                     ,rotation_range=45
                                     ,width_shift_range=0.2
                                     ,height_shift_range=0.2
                                     ,shear_range=0.2
                                     ,zoom_range=0.25
                                     ,horizontal_flip=True
                                     ,fill_mode='nearest'
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

In [ ]:
NUM_CLASSES = train_batches.num_classes

In [ ]:
base_model = keras.applications.mobilenet.MobileNet()

In [ ]:
?keras.applications.mobilenet.MobileNet()

In [ ]:
base_model.summary()

In [ ]:
x = base_model.layers[-6].output
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.summary()

In [ ]:
model.layers[:-1]

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    

In [ ]:
SVG(model_to_dot(model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))


In [ ]:
#model.summary()

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_batches, steps_per_epoch=train_batches.n//BATCH_SIZE, 
                    validation_data=valid_batches
                    ,  validation_steps=valid_batches.n // BATCH_SIZE, verbose=2,workers=4, 
                   use_multiprocessing=True
                    , epochs=3
                   
                   )


In [ ]:
train_batches.class_indices.keys()

In [ ]:
train_batches.class_indices

In [ ]:
model.save('dog_breed.h5')

In [ ]:
with open('labels.txt', 'w') as file_handler:
    for item in train_batches.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [8]:
with tf.Session() as sess:

    custom_objects={'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}

    model = keras.models.load_model('dog_breed.h5',custom_objects=custom_objects)

In [13]:
with tf.Session() as sess:
    custom_objects={'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}

    model = keras.models.load_model('dog_breed.h5',custom_objects=custom_objects)
    
    signature = tf.saved_model.signature_def_utils.predict_signature_def(                                                                        
    inputs={'image': model.input}, outputs={'scores': model.output})                                                                         

    path = "prod_models"
    shutil.rmtree(path,ignore_errors=True)
    builder = tf.saved_model.builder.SavedModelBuilder(path) 
    
    builder.add_meta_graph_and_variables(                                                                                                        
            sess=sess,                                                                                                                    
            tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
            signature_def_map={                                                                                                                      
                tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:                                                                
                    signature                                                                                                                        
            })                                                                                                                                       
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.


FailedPreconditionError: Attempting to use uninitialized value conv1/kernel
	 [[Node: conv1/kernel/_1592 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_30_conv1/kernel", _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1/kernel)]]
	 [[Node: conv_pw_2_bn_4/moving_variance/local_step/_4079 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1273_conv_pw_2_bn_4/moving_variance/local_step", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](^save_1/ShardedFilename, ^save_1/SaveV2/tensor_names, ^save_1/SaveV2/shape_and_slices)]]

In [ ]:
from keras import backend as K                                                                                                               
import tensorflow as tf                                                                                                                      
 
    
signature = tf.saved_model.signature_def_utils.predict_signature_def(                                                                        
    inputs={'image': model.input}, outputs={'scores': model.output})                                                                         

path = "prod_models"
shutil.rmtree(path,ignore_errors=True)
builder = tf.saved_model.builder.SavedModelBuilder(path)                                                                    

with tf.Session() as sess:

    builder.add_meta_graph_and_variables(                                                                                                        
        sess=sess,                                                                                                                    
        tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
        signature_def_map={                                                                                                                      
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:                                                                
                signature                                                                                                                        
        })                                                                                                                                       
    builder.save()

In [ ]:
!ls prod_models/saved_model.pb

In [ ]:
!ls

In [ ]:
!pwd

In [30]:
?tf.contrib.lite.TocoConverter.from_frozen_graph

In [ ]:
?tf.contrib.lite.TocoConverter.from_saved_model

In [ ]:
tf.contrib.lite.TocoConverter.from_frozen_graph

In [21]:
l = model.input_layers[0]

In [22]:
l.name

'input_1'

In [29]:
model.outputs[-1]

<tf.Tensor 'dense_1_6/Softmax:0' shape=(?, 120) dtype=float32>

In [25]:
model.output_layers[-2].name

IndexError: list index out of range

In [33]:
!ls ../

deployment  dog_breed.h5.pb  keras_to_tensorflow  prod_models  setup  training


In [35]:
tf.contrib.lite.TocoConverter.from_frozen_graph(graph_def_file='../dog_breed.h5.pb'
                                                , input_arrays = ['input_1']
                                                ,output_arrays=['output_node0'])

ValueError: Please freeze the graph using freeze_graph.py

In [15]:
tf.contrib.lite.TocoConverter.from_saved_model('prod_models')

OSError: SavedModel file does not exist at: prod_models

In [ ]:
!pwd

In [ ]:
tra

In [ ]:
!gsutil cp /home/ubuntu/custom/deeplizrd/dog_breed.h5 gs://np-training-public/

In [ ]:
!pwd

In [ ]:

!ls /home/ubuntu/.kaggle/competitions/dog-breed-identification/train_folder/

